In [1]:
from htmd import *
htmd.config(viewer='ngl')
#os.chdir('/webdata/73hboiwia98hdj209jq0/')  # Skip this command.


Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049

You are on the latest HTMD version (unpackaged : /home/joao/maindisk/software/repos/Acellera/htmd/htmd).



# System building: trypsin-benzamidine

## Obtain the structures

(Download the required files for the tutorial from this [link](http://pub.htmd.org/73hboiwia98hdj209jq0/bentryp.tar.gz). )

In [2]:
prot = Molecule('3PTB')

2017-01-10 23:46:20,409 - htmd.molecule.molecule - INFO - Using local copy for 3PTB: /home/joao/maindisk/software/repos/Acellera/htmd/htmd/data/pdb/3ptb.pdb


## Clean the structures

Water molecules and one calcium ion present in the PDB crystal structure were also read by HTMD. Start by removing the ligand:

In [3]:
prot.remove('resname BEN')

2017-01-10 23:46:23,612 - htmd.molecule.molecule - INFO - Removed 9 atoms. 1692 atoms remaining in the molecule.


array([1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638], dtype=int32)

## Preparing the protein

In [4]:
prot = proteinPrepare(prot)

2017-01-10 23:46:27,597 - propka - INFO - No pdbfile provided
2017-01-10 23:46:30,839 - htmd.builder.preparation - WARNING - The following residue has not been optimized: CA
2017-01-10 23:46:37,350 - htmd.builder.preparationdata - INFO - The following residues are in a non-standard state: CYS    22  A (CYX), HIS    40  A (HIE), CYS    42  A (CYX), HIS    57  A (HIP), CYS    58  A (CYX), HIS    91  A (HID), CYS   128  A (CYX), CYS   136  A (CYX), CYS   157  A (CYX), CYS   168  A (CYX), CYS   182  A (CYX), CYS   191  A (CYX), CYS   201  A (CYX), CYS   220  A (CYX), CYS   232  A (CYX)
2017-01-10 23:46:37,356 - htmd.builder.preparationdata - WARNING - Dubious protonation state: the pKa of 4 residues is within 1.0 units of pH 7.0.
2017-01-10 23:46:37,359 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS    57  A (pKa= 7.46)
2017-01-10 23:46:37,361 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    GLU    70  A (pKa= 6.10)
2017-01-10 23:46:37,

## Define segments

To build a system in HTMD, we need to separate the chemical molecules into separate segments. This prevents the builder from accidentally bonding different chemical molecules and allows us to add caps to them.

In [5]:
prot.set('segid', 'P', sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')

Center the protein to the origin

In [6]:
prot.center()

## Let's add a ligand!

In [7]:
ligand = Molecule('bentryp/benzamidine.pdb')
ligand.center()
ligand.view(sel='resname MOL', style='Licorice')

But the ligand is located inside the protein...
We would like the ligand to be:

* At a certain distance from the protein
* Rotated randomly, to provide different starting conditions

## Let's reposition the ligand then

In [8]:
ligand.rotateBy(uniformRandomRotation())

This took care of the ligand rotation around its center. 
We still need to position it far from the protein.
First, find out the radius of the protein:

![](http://pub.htmd.org/73hboiwia98hdj209jq0/maxdist.png)

In [9]:
from htmd.molecule.util import maxDistance
D = maxDistance(prot, 'all')
print(D)

28.832803412


In [10]:
D += 10
ligand.moveBy([D, 0, 0])  # Move the ligand 10 Angstrom away from the furthest protein atom in X dimension
ligand.rotateBy(uniformRandomRotation())

## Don't forget to set segments

In [11]:
ligand.set('segid','L')
ligand.set('resname','MOL')

### Mix it all together

In [12]:
mol = Molecule(name='combo')
mol.append(prot)
mol.append(ligand)
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='resname MOL', style='Licorice')
mol.view()

## Solvate

> Water is the driving force of all nature. --Leonardo da Vinci

![](http://pub.htmd.org/73hboiwia98hdj209jq0/waterbox.png)

In [13]:
D = D + 5
smol = solvate(mol, minmax=[[-D, -D, -D], [D, D, D]])
smol.reps.add(sel='water', style='Lines')
smol.view()

2017-01-10 23:48:01,252 - htmd.builder.solvate - INFO - Using water pdb file at: /data/joao/maindisk/software/repos/Acellera/htmd/htmd/builder/wat.pdb
2017-01-10 23:48:02,236 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2


Solvating: 100% (8/8) [############################################] eta 00:00 /


2017-01-10 23:48:23,595 - htmd.builder.solvate - INFO - After removing water molecules colliding with other molecules, 20149 water molecules were added to the system.


## Available CHARMM forcefields

In [14]:
charmm.listFiles()

---- Topologies files list: /home/joao/maindisk/software/repos/Acellera/htmd/htmd/builder/charmmfiles/top/ ----
top/top_all22_prot.rtf
top/top_all22star_prot.rtf
top/top_all35_ethers.rtf
top/top_all36_carb.rtf
top/top_all36_cgenff.rtf
top/top_all36_lipid.rtf
top/top_all36_na.rtf
top/top_all36_prot.rtf
top/top_water_ions.rtf
---- Parameters files list: /home/joao/maindisk/software/repos/Acellera/htmd/htmd/builder/charmmfiles/par/ ----
par/par_all22_prot.prm
par/par_all22star_prot.prm
par/par_all35_ethers.prm
par/par_all36_carb.prm
par/par_all36_cgenff.prm
par/par_all36_lipid.prm
par/par_all36_na.prm
par/par_all36_prot.prm
par/par_all36_prot_mod.prm
par/par_water_ions.prm
---- Stream files list: /home/joao/maindisk/software/repos/Acellera/htmd/htmd/builder/charmmfiles/str/ ----
str/carb/toppar_all36_carb_glycolipid.str
str/carb/toppar_all36_carb_glycopeptide.str
str/carb/toppar_all36_carb_model.str
str/lipid/toppar_all36_lipid_bacterial.str
str/lipid/toppar_all36_lipid_cardiolipin.str
st

## Build and ionize using CHARMM

In [15]:
topos  = ['top/top_all22star_prot.rtf', 'bentryp/benzamidine.rtf']
params = ['par/par_all22star_prot.prm', 'bentryp/benzamidine.prm']

bmol = charmm.build(smol, topo=topos, param=params, 
                        outdir='./build')

2017-01-10 23:50:21,438 - htmd.builder.charmm - INFO - Writing out segments.


Bond between A: [serial 92 resid 22 resname CYX chain A segid P]
             B: [serial 1988 resid 157 resname CYX chain A segid P]

Bond between A: [serial 351 resid 42 resname CYX chain A segid P]
             B: [serial 571 resid 58 resname CYX chain A segid P]

Bond between A: [serial 1604 resid 128 resname CYX chain A segid P]
             B: [serial 3004 resid 232 resname CYX chain A segid P]

Bond between A: [serial 1683 resid 136 resname CYX chain A segid P]
             B: [serial 2611 resid 201 resname CYX chain A segid P]

Bond between A: [serial 2146 resid 168 resname CYX chain A segid P]
             B: [serial 2353 resid 182 resname CYX chain A segid P]

Bond between A: [serial 2494 resid 191 resname CYX chain A segid P]
             B: [serial 2799 resid 220 resname CYX chain A segid P]



2017-01-10 23:51:20,218 - htmd.builder.builder - INFO - 6 disulfide bonds were added
2017-01-10 23:51:20,356 - htmd.builder.charmm - INFO - Starting the build.
2017-01-10 23:51:21,098 - htmd.builder.charmm - WARNING - Failed to set coordinates for 8 atoms.
2017-01-10 23:51:21,101 - htmd.builder.charmm - WARNING - Failed to guess coordinates for 2 atoms due to bad angles.
2017-01-10 23:51:21,104 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 15 atoms.
2017-01-10 23:51:21,105 - htmd.builder.charmm - WARNING - Please check /data/joao/maindisk/SANDBOX/03_PreparationBuilding_TGiorgino/build/log.txt for further information.
2017-01-10 23:51:21,106 - htmd.builder.charmm - INFO - Finished building.
2017-01-10 23:51:25,304 - htmd.builder.ionize - INFO - Adding 10 anions + 0 cations for neutralizing and 0 ions for the given salt concentration.
2017-01-10 23:51:25,915 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2017-01-10 23:51:25,917 - htmd.builder.ion

## Visualize

In [16]:
bmol.view()